#  Tone Classification Task

## Data Preparation

In [50]:
require 'loader'
MAX_LEN = 128
-- Loader:new(obj, max_len)
ldr = Loader:new(nil, MAX_LEN)

In [51]:
function make_dataset(name)
    local dataset = {data = ldr.data[name], label = ldr.label[name]}
    function dataset:size() 
        return self.data:size(1) 
    end
    for i = 1, dataset:size() do
        dataset[i] = {dataset.data[i], dataset.label[i]}
    end
    for i = 1, 2 do
--         we abandon engy...
        if i == 2 then
            dataset.data[{{}, {}, i}]:fill(0)
            break
        end
        local std = dataset.data[{{}, {}, i}]:std()
        dataset.data[{{}, {}, i}]:div(std)
--         for j = 1, dataset:size() do
--             local std = dataset.data[{j, {}, i}]:std()
--             dataset.data[{j, {}, i}]:div(std)
--         end
    end
    return dataset
end

In [52]:
trainset = make_dataset('train')
testset = make_dataset('test')
newtestset = make_dataset('test_new')

In [53]:
Plot = require 'itorch.Plot'
local x1 = torch.range(1,MAX_LEN)
local sampleN = 4
local y1 = newtestset.data[{sampleN,{},1}]
local y2 = newtestset.data[{sampleN,{},2}]
plot = Plot():line(x1, y1,'red','f0'):legend(true):title('Feature Observation of a Tone '..newtestset.label[sampleN]):draw()
plot:line(x1, y2,'blue','engy'):redraw()

## Model

In [54]:
require 'dp'
local inp = 2;  -- dimensionality of one sequence element 
local outp = 4; -- number of derived features for one sequence element
local conv_kw = 4;   -- number of sequence element kernel operates on per step
local conv_dw = 4;   -- step dw and go on to the next sequence element

local pool_kw = 4;   -- number of sequence element pooled on per step
local pool_dw = 4;   -- step dw and go on to the next sequence element

local nOutputFrame = (MAX_LEN - conv_kw) / conv_dw + 1
local nOutputFrame = (nOutputFrame - pool_kw) / pool_dw + 1

net = nn.Sequential()
peek = nn.TemporalConvolution(inp, outp, conv_kw, conv_dw)
net:add(peek)
net:add(nn.ReLU())
net:add(nn.TemporalMaxPooling(pool_kw, pool_dw))
net:add(nn.View(outp*nOutputFrame))
net:add(nn.Linear(outp*nOutputFrame, 64))
-- net:add(nn.BatchNormalization(64))
net:add(nn.ReLU())
net:add(nn.Linear(64, 4))
net:add(nn.LogSoftMax())

In [55]:
print (net:__tostring())

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> output]
  (1): nn.TemporalConvolution
  (2): nn.ReLU
  (3): nn.TemporalMaxPooling
  (4): nn.View(32)
  (5): nn.Linear(32 -> 64)
  (6): nn.ReLU
  (7): nn.Linear(64 -> 4)
  (8): nn.LogSoftMax
}	


In [56]:
criterion = nn.ClassNLLCriterion();  
trainer = nn.StochasticGradient(net, criterion)  
trainer.learningRate = 0.001
trainer.maxIteration = 200
trainer.verbose = False

In [57]:
function cal_acc(dataset)
    local acc = 0.0
    for i = 1, dataset:size() do
        _, output = torch.max(net:forward(dataset[i][1]), 1)
        if output[1] == dataset[i][2] then
            acc = acc + 1.0
        end
    end
    acc = acc / dataset:size()
    return acc
end

In [58]:
recorder = {loss = {}, trainacc = {}, valacc = {}, testacc = {}}
trainer.hookIteration = function(obj, iteration, currentError) 
    local trainacc = cal_acc(trainset)
    local valacc = cal_acc(testset)
    local testacc = cal_acc(newtestset)
    table.insert(recorder.loss, currentError)
    table.insert(recorder.trainacc, trainacc)
    table.insert(recorder.valacc, valacc)
    table.insert(recorder.testacc, testacc)
    if (iteration % 10 == 0) then 
        print('Iteration '..iteration..': loss('..currentError..') train_acc('
            ..trainacc..') test_acc('..valacc..') new_test_acc('..testacc..').')
    end
end

In [59]:
trainer:train(trainset)

# StochasticGradient: training	


Iteration 10: loss(0.58058444184859) train_acc(0.81) test_acc(0.825) new_test_acc(0.59649122807018).	


Iteration 20: loss(0.38598727272817) train_acc(0.88) test_acc(0.85) new_test_acc(0.66666666666667).	


Iteration 30: loss(0.30057962028006) train_acc(0.93) test_acc(0.9) new_test_acc(0.73684210526316).	


Iteration 40: loss(0.26742195250152) train_acc(0.9325) test_acc(0.925) new_test_acc(0.76315789473684).	


...

Iteration 160: loss(0.13665207386385) train_acc(0.975) test_acc(0.975) new_test_acc(0.80701754385965).	


Iteration 170: loss(0.13184267851597) train_acc(0.9775) test_acc(0.975) new_test_acc(0.81140350877193).	


Iteration 180: loss(0.12836403703631) train_acc(0.9775) test_acc(0.975) new_test_acc(0.81140350877193).	


Iteration 190: loss(0.12537567936725) train_acc(0.9775) test_acc(0.975) new_test_acc(0.81140350877193).	


Iteration 200: loss(0.12258618229321) train_acc(0.9775) test_acc(0.975) new_test_acc(0.81140350877193).	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.12258618229321	



In [60]:
local x = torch.range(1, #recorder.loss)
plt = Plot():line(x, recorder.loss,'red','loss'):legend(true):title('Train Loss'):draw()
plt = Plot():line(x, recorder.trainacc,'blue','train_acc'):legend(true):title('Accuracy'):draw()
plt:line(x, recorder.valacc,'green','val_acc'):redraw()

In [66]:
print ('accuracy on train set '..cal_acc(trainset))
print ('accuracy on test set '..cal_acc(testset))
print ('accuracy on new test set '..cal_acc(newtestset))

accuracy on train set 0.9775	


accuracy on test set 0.975	


accuracy on new test set 0.81140350877193	


## To see why

In [77]:
mislabeled = {}
local output
for i = 1, newtestset:size() do
    _, output = torch.max(net:forward(newtestset[i][1]), 1)
    if output[1] ~= newtestset[i][2] then
        table.insert(mislabeled, { data_id = i, pred = output[1], gt = newtestset[i][2]})
    end
end

In [78]:
print(#mislabeled)

43	


In [79]:
print(mislabeled)

{
  1 : 
    {
      data_id : 1
      pred : 3
      gt : 1
    }
  2 : 
    {
      data_id : 3
      pred : 3
      gt : 1
    }
  3 : 
    {
      data_id : 4
      pred : 3
      gt : 1
    }
  4 : 
    {
      data_id : 5
      pred : 3
      gt : 1
    }
  5 : 
    {
      data_id : 6
      pred : 3
      gt : 1
    }
  6 : 
    {
      data_id : 7
      pred : 3
      gt : 1
    }
  7 : 
    {
      data_id : 8
      pred : 3
      gt : 1
    }
  8 : 
    {
      data_id : 9
      pred : 3
      gt : 1
    }
  9 : 
    {
      data_id : 10
      pred : 3
      gt : 1
    }
  10 : 
    {
      data_id : 11
      pred : 3
      gt : 1
    }
  11 : 
    {
      data_id : 12
      pred : 3
      gt : 1
    }
  12 : 
    {
      data_id : 18
      pred : 3
      gt : 1
    }
  13 : 
    {
      data_id : 20
      pred : 2
      gt : 1
    }
  14 : 
    {
      data_id : 26
      pred : 3
      gt : 1
    }
  15 : 
    {
      data_id : 28
      pred : 3
      gt : 1
    }
  16 : 
  

  17 : 
    {
      data_id : 37
      pred : 2
      gt : 1
    }
  18 : 
    {
      data_id : 41
      pred : 2
      gt : 1
    }
  19 : 
    {
      data_id : 56
      pred : 3
      gt : 2
    }
  20 : 
    {
      data_id : 59
      pred : 4
      gt : 2
    }
  21 : 
    {
      data_id : 62
      pred : 3
      gt : 2
    }
  22 : 
    {
      data_id : 91
      pred : 1
      gt : 2
    }
  23 : 
    {
      data_id : 93
      pred : 1
      gt : 2
    }
  24 : 
    {
      data_id : 95
      pred : 1
      gt : 2
    }
  25 : 
    {
      data_id : 96
      pred : 1
      gt : 2
    }
  26 : 
    {
      data_id : 101
      pred : 4
      gt : 2
    }
  27 : 
    {
      data_id : 105
      pred : 1
      gt : 2
    }
  28 : 
    {
      data_id : 108
      pred : 1
      gt : 2
    }
  29 : 
    {
      data_id : 109
      pred : 1
      gt : 2
    }
  30 : 
    {
      data_id : 114
      pred : 1
      gt : 2
    }
  31 : 
    {
      data_id : 119
      pred : 1
      gt

    }
  39 : 
    {
      data_id : 195
      pred : 1
      gt : 4
    }
  40 : 
    {
      data_id : 198
      pred : 1
      gt : 4
    }
  41 : 
    {
      data_id : 222
      pred : 1
      gt : 4
    }
  42 : 
    {
      data_id : 225
      pred : 1
      gt : 4
    }
  43 : 
    {
      data_id : 228
      pred : 1
      gt : 4
    }
}


In [65]:
print(net:forward(newtestset[4][1]))
-- print(peek:forward(newtestset[4][1]))
print(peek.weight)
-- local x1 = torch.range(1, MAX_LEN - 1)
-- local sampleN = 7
-- local y1 = newtestset.data[{sampleN,{},1}]
-- local y2 = peek
-- plot = Plot():line(x1, y1,'red','f0'):legend(true):title('Feature Observation of a Tone '..newtestset.label[sampleN]):draw()
-- plot:line(x1, y2,'blue','engy'):redraw()

-10.3447
 -5.6717
 -0.0035
-13.9952
[torch.DoubleTensor of size 4]

 0.0278  0.2201 -0.3358  0.2727 -0.0591  0.3355  0.1564 -0.3124
 1.2147  0.2346  1.0240  0.2417  0.7668 -0.2452  0.7629 -0.2245
 0.6831 -0.3430  0.6562  0.3469  0.5395  0.3016  0.4841 -0.0978
 0.2870 -0.2614  0.7241 -0.2310  0.8362 -0.0810  0.2077  0.3248
[torch.DoubleTensor of size 4x8]

